In [1]:
import sys
import time
import urllib
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
#1
import urllib.request, sys, time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet = 5

all_data = []  # Create an empty list to store all the data

for page in range(1, pagesToGet + 1):
    print('processing page:', page)
    url = 'https://www.newsinlevels.com/' + str(page)
    print(url)

    # an exception might be thrown, so the code should be in a try-except block
    try:
        # use the browser to get the url. This is a suspicious command that might blow up.
        page = requests.get(url)  # this might throw an exception if something goes wrong.

    except Exception as e:  # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()  # get the exception information
        print('ERROR FOR LINK:', url)  # print the link that caused the problem
        print(error_type, 'Line:', error_info.tb_lineno)  # print error info and line that threw the exception
        continue  # ignore this page. Abandon this and go back.

    time.sleep(2)
    soup = BeautifulSoup(page.text, 'html.parser')
    links = soup.find_all('li', attrs={'class': 'o-listicle__item'})
    print(len(links))

    for j in links:
        link = "https://www.newsinlevels.com/"
        link += j.find("div", attrs={'class': 'm-statement__quote'}).find('a')['href'].strip()

        # Now, extract data from the inner link
        inner_page = requests.get(link)
        inner_soup = BeautifulSoup(inner_page.text, 'html.parser')

        statement = inner_soup.find("div", attrs={'class': 'm-statement__quote'}).text.strip()
        
        # Check if the 'footer' element exists before accessing its 'text' attribute
        date_element = inner_soup.find('div', attrs={'class': 'm-statement__body'}).find('footer')
        date = date_element.text[-14:-1].strip() if date_element else 'N/A'
        
        source = inner_soup.find('div', attrs={'class': 'm-statement__meta'}).find('a').text.strip()
        label = inner_soup.find('div', attrs={'class': 'm-statement__content'}).find('img',
                                                                                        attrs={'class': 'c-image__original'}).get(
            'alt').strip()

        all_data.append((statement, link, date, source, label))

# Convert all_data to a DataFrame
data_df = pd.DataFrame(all_data, columns=['Statement', 'Link', 'Date', 'Source', 'Label'])

# Save the data to a CSV file
csv_filename = "inner_data.csv"
data_df.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"All inner data has been saved to {csv_filename}")


processing page: 1
https://www.politifact.com/factchecks/list/?page=1
30
processing page: 2
https://www.politifact.com/factchecks/list/?page=2
30
processing page: 3
https://www.politifact.com/factchecks/list/?page=3
30
processing page: 4
https://www.politifact.com/factchecks/list/?page=4
30
processing page: 5
https://www.politifact.com/factchecks/list/?page=5
30
All inner data has been saved to inner_data.csv


In [4]:
df = pd.read_csv('inner_data.csv')
df.head()

,Statement,Link,Date,Source,Label
0,"Videos of Nancy Pelosi from Jan. 6, 2021, show...",https://www.politifact.com/factchecks/2023/nov...,NaN,Instagram posts,false
1,“Israel has changed their barcode from 729 to ...,https://www.politifact.com/factchecks/2023/nov...,NaN,TikTok posts,false
2,Video shows that “Troy Nehls FORCES Pete Butti...,https://www.politifact.com/factchecks/2023/nov...,NaN,Facebook posts,false
3,"“Because wages are rising, this Thanksgiving d...",https://www.politifact.com/factchecks/2023/nov...,NaN,Karine Jean-Pierre,mostly-true
4,“New police bodycam footage” of George Floyd’s...,https://www.politifact.com/factchecks/2023/nov...,NaN,Megyn Kelly,false


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load scraped data
df = pd.read_csv('inner_data.csv')

# Split the data for testing
X_train, X_test, y_train, y_test = train_test_split(df['Statement'], df['Label'], test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the model
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Make predictions
predictions = model.predict(X_test_vec)

# Evaluate and report accuracy
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

# Store evaluation report in CSV
evaluation_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
evaluation_df.to_csv('evaluation_report.csv', index=False)


Accuracy: 0.6333333333333333
Classification Report:
               precision    recall  f1-score   support

 barely-true       0.00      0.00      0.00         4
       false       0.64      0.95      0.77        19
 mostly-true       0.00      0.00      0.00         1
  pants-fire       0.50      0.20      0.29         5
        true       0.00      0.00      0.00         1

    accuracy                           0.63        30
   macro avg       0.23      0.23      0.21        30
weighted avg       0.49      0.63      0.53        30



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.